<a href="https://colab.research.google.com/github/Transega/Smile-randomforest-supervised-classification/blob/main/Image_collection_and_Supervised_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Installs geemap package
import subprocess
 
try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
 
# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap
 
# Authenticates and initializes Earth Engine
import ee
 
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=wrdG_Gk10JswPm1QY5p7eIqE8mlaOXh7LuKtYyrWlJk&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4gAbakKtQrVcbcGCBNM93CCiIohw4iFzAjmBxqimaZefmJ5oEJSp8

Successfully saved authorization token.


In [ ]:
# !pip install geopandas 
# !pip -q install rasterio

# 1 Import Libraries

In [6]:
import ee
import geemap
import geemap.eefolium
#import rasterio
#from rasterio import plot
#%matplotlib inline

# 2 Define Area of interest

Here replace the counties affected with your google earth engine asset or use a shapefile and convert it to GEE asset using the function in section *3* you may also subset the various counties. 

In [7]:
# Counties Affected Area of interest
Counties_affected =  ee.FeatureCollection('users/collinsasegaca/Affected_counties') #https://drive.google.com/file/d/1VAZ879vzOUv9Hbo2PWFwCczYIGWityfS/view?usp=sharing
 
# Subset the counties since the entire region takes too much time to compute  #link to the shapefiles https://drive.google.com/drive/folders/1eFN6ZZKN3f-KHJMx4AaDP2Ko3psUOb6W?usp=sharing
Elgeiyo_Baringo = './drive/MyDrive/omdena/affected_counties/brokendown/Elgeyo_baringo.shp' # Elgeiyo Marakwet & Baringo 
Laikipia_samburu = './drive/MyDrive/omdena/affected_counties/brokendown/Laikipia_samburu.shp' #  Laikipia_samburu
Kitui = './drive/MyDrive/omdena/affected_counties/brokendown/Kitui.shp' # Kitui
Mandera = './drive/MyDrive/omdena/affected_counties/brokendown/Mandera.shp' # Mandera
Machakos_embu = './drive/MyDrive/omdena/affected_counties/brokendown/Machakos_embu.shp' # Machakos_embu
Marsabit = './drive/MyDrive/omdena/affected_counties/brokendown/Marsabit.shp' # Marsabit
Meru_isolo ='./drive/MyDrive/omdena/affected_counties/brokendown/Meru_isolo.shp' # Meru and Isiolo
Tana = './drive/MyDrive/omdena/affected_counties/brokendown/Tana.shp' #Tana
Turkana = './drive/MyDrive/omdena/affected_counties/brokendown/Turkana.shp' 
Wajir = './drive/MyDrive/omdena/affected_counties/brokendown/Wajir.shp' #Wajir 
Garissa = './drive/MyDrive/omdena/affected_counties/brokendown/Garissa.shp' # Garisa

 
# 

3 Convert shapefiles to google earth engine objects 

In [8]:
def conv(in_shp):
  ee_object = geemap.shp_to_ee(in_shp)
  return ee_object

4 Define the dates

In [9]:
# date before attack
fdate = '2019-08-1' #('2017-01-1' ,'2017-12-31')
ldate = '2019-12-25'
 
#date after attack
after_fdate = '2020-1-1'
after_ldate = '2020-5-31'

# 5 Image collection

In [10]:
def sentinel_2_before(image):
  collection = ee.ImageCollection("COPERNICUS/S2") \
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10)) \
    .filterBounds(geometry) \
 
  before_collection = collection.filterDate(fdate,ldate)# before attack
  
  medianpixels_before = before_collection.median()
  medianpixelsclipped_before = medianpixels_before.clip(geometry).divide(10000)
  
 
  vis_params = {
    'min': 0,
    'max': 1,
    'bands': ['B12', 'B11', 'B4'], 'gamma': 1.5
  }
  return medianpixelsclipped_before
 
# get after images 
def sentinel_2_After(image):
  collection = ee.ImageCollection("COPERNICUS/S2") \
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10)) \
    .filterBounds(geometry) \
 
  after_collection = collection.filterDate(after_fdate,after_ldate)# after attack
 
  medianpixels_after = after_collection.median()
  medianpixelsclipped_after = medianpixels_after.clip(geometry).divide(10000) 
 
 
  vis_params = {
    'min': 0,
    'max': 1,
    'bands': ['B12', 'B11', 'B4'], 'gamma': 1.5
  }
  return medianpixelsclipped_after

In [ ]:
# Get before image for all counties affected 
geometry = Counties_affected 
All_counties_before = sentinel_2_before(geometry)

In [ ]:
# Get after Images for all affected counties 
All_Counties_after = sentinel_2_After(geometry)
# Visualization parameters
vis_params = {
    'min': 0,
    'max': 1,
    'bands': ['B12', 'B11', 'B4'], 'gamma': 1.5
  }

6 Visualize the Image

In [ ]:
Map_p = geemap.Map()
Map_p.addLayer(All_counties_before, vis_params, 'Affected Counties Sentinel 2 Before attack image')
Map_p.addLayer(All_Counties_after, vis_params, 'Affected  Counties Sentinel 2 After attack image')
Map_p.centerObject(Counties_affected, 8)
Map_p.addLayerControl()
Map_p

7 Build a webmap visualization 


In [ ]:
Map_p.save('counties_affected_before&after.html',close_file=True)

Getting images for respective counties 
here you only read to read a shapefile from your drive and use the function to convert to gee object 

In [ ]:
AOI = conv(Garissa) # converts Garrisa county shapefile to gee object
 
Garisa_image = All_counties_before.clip(AOI.geometry()) # obtains  cliped image for the county
 


In [ ]:
# visualize Garrisa image
Map_p = geemap.Map()
Map_p.addLayer(Garisa_image, vis_params, 'Garissa Sentinel 2 Before image')
Map_p.centerObject(AOI, 8)
Map_p.addLayerControl()
Map_p

Generate training points from sentinel 2 2016 LULC image for kenya 

link= http://geoportal.rcmrd.org/layers/servir%3Akenya_sentinel2_lulc2016

In [ ]:
# Make the training dataset.
def training(points):
  S2LULC_2016 = ee.Image('users/collinsasegaca/Kenya_Sentinel2_LULC2016').select('b1').clip(ROI.geometry())
  points = S2LULC_2016.sample(**{
      'region': ROI.geometry(),
      'scale': 10,
      'numPixels': 4000,
      'seed': 0,
      'geometries': True  # Set this to False to ignore geometries
  })
  return points

In [ ]:
ROI = AOI

In [ ]:
points_garissa = training(ROI) #generation of Random points for training our model for classification 

In [ ]:
#Visualization of the training points
Map3 = geemap.Map()
Map3.centerObject(ROI,8)
Map3.addLayer(points_garissa.draw("blue"), {}, 'training', False);
Map3

8 Train the model and classify the images

In [ ]:
# Training and classifiy
def Train_classify(trainin_data): 
  bands = ['B2', 'B3', 'B4', 'B8']
 
 
# This property of the table stores the land cover labels.
  label = 'b1'
 
# Overlay the points on the imagery to get training for the before imge
  training = image.select(bands).sampleRegions(**{
    'collection': points,
    'properties': [label],
    'scale': 10
  })
 
# Train a CART classifier with default parameters.
  trained = ee.Classifier.smileCart().train(training, label, bands)
  # Classify the image with the same bands used for training.
  result = image.select(bands).classify(trained)
  return result

In [ ]:
image = Garisa_image
points = points_garissa

In [ ]:
Garrisa_Classified_before = Train_classify(image) #Classify the before attack  Image 

In [ ]:
#Export result
geemap.ee_export_image_to_drive(Garrisa_Classified_before, description='Garissa_before', folder='omdena', scale=10)

In [ ]:
#after
AOI = conv(Garissa) # define AOI for the county
Garissa_image_after = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Garissa = training(ROI) #this generates points from the function

# Training and classifiy

image = Garissa_image_after # defines the image for the Location
points = points_garissa # defines the points to be used 
Garissa_Classified_After = Train_classify(image)

## 9 Accuracy assement 
## Overall Accuracy essentially tells us out of all of the reference sites what proportion were mapped correctly. The overall accuracy is usually expressed as a percent, with 100% accuracy being a perfect classification where all reference site were classified correctly

There are various methods to check accuracy including:

***Kappa***
The Kappa Coefficient is generated from a statistical test to evaluate the accuracy of a classification. Kappa essentially evaluates how well the classification performed as compared to just randomly assigning values, i.e. did the classification do better than random. The Kappa Coefficient can range from -1 t0 1. A value of 0 indicated that the classification is no better than a random classification. A negative number indicates the classification is significantly worse than random. A value close to 1 indicates that the classification is significantly better than random.

***confusionMatrix()***

# confusionMatrix() computes a 2D confusion matrix for a classifier based on its training data (ie: resubstitution error). Axis 0 of the matrix correspond to the input classes (i.e., reference data), and axis 1 to the output classes (i.e., classification data). The rows and columns start at class 0 and increase sequentially up to the maximum class value





In [ ]:
#Split Training and testing
def train_test(train_Test):
  bands = ['B2', 'B3', 'B4', 'B8']


  # This property of the table stores the land cover labels.
  label = 'b1'

  # Overlay the points on the imagery to get training for the before imge
  sample = image.select(bands).sampleRegions(**{
    'collection': points,
    'properties': [label],
    'scale': 10
  })

  # Adds a column of deterministic pseudorandom numbers. 
  sample = sample.randomColumn()

  split = 0.7

  training_dta = sample.filter(ee.Filter.lt('random', split))
  validation_dta = sample.filter(ee.Filter.gte('random', split))
  #inspect the data
  training_dta.first().getInfo()
  validation_dta.first().getInfo()
  # Train Classifier
  classifier = ee.Classifier.smileRandomForest(10).train(training_dta, label, bands)
  # Classify the image with the same bands used for training.
  result_1 = image.select(bands).classify(classifier)
  train_accuracy = classifier.confusionMatrix()
  train_accuracy.getInfo()
  train_accuracy.accuracy().getInfo()
  print('confusionMatrix:',train_accuracy.accuracy().getInfo())
  print('Kappa:',train_accuracy.kappa().getInfo())
  validated = validation_dta.classify(classifier) # use the validation dataset
  #test accuracy
  test_accuracy = validated.errorMatrix('b1', 'classification') # errorMatrix computes a 2D error matrix for a collection by comparing two columns of a collection: one containing the actual values, and one containing predicted values.
  test_accuracy.accuracy().getInfo()
  return test_accuracy

In [ ]:
image = Garisa_image
points = points_garissa



In [ ]:
accuracy_Garisaa = train_test(image)

confusionMatrix: 0.9308336366945759
Kappa: 0.8783109627849728


Elgeiyo Markwet and Baringo county

In [ ]:

AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Elgyo_Baringo_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Elgyo_Baringo_image # defines the image for the Location
points = points_Elgeiyo_baringo # defines the points to be used 
Elgyo_Baringo_Classified_before = Train_classify(image)

#after
AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Elgyo_Baringo_image_after = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Elgyo_Baringo_image_after # defines the image for the Location
points = points_Elgeiyo_baringo # defines the points to be used 
Elgyo_Baringo_Classified_After = Train_classify(image)





Accuracy asessment 

In [ ]:
image = Elgyo_Baringo_image
points = points_Elgeiyo_baringo

In [ ]:
accuracy_Elgyo_Baringo = train_test(image)


confusionMatrix: 0.929523141654979
Kappa: 0.9045890628240341


In [ ]:
#Export Before and After Images for 
# geemap.ee_export_image_to_drive(Elgyo_Baringo_Classified_After, description='Elgeiyo_Baringo_After', folder='omdena', scale=10)
# geemap.ee_export_image_to_drive(Elgyo_Baringo_Classified_before, descripti='Elgeiyo_Baringo_before', folder='omdena', scale=10)

Kitui

In [ ]:
AOI = conv(Kitui) # define AOI for the county
Kitui_image_before = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Kitui = training(ROI) #this generates points from the function


# Training and classifiy

image = Kitui_image_before # defines the image for the Location
points = points_Kitui # defines the points to be used 
Kitui_Classified_before = Train_classify(image)



In [ ]:
#after 
#AOI = conv(E) # define AOI for the county
Kitui_after = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Kitui = training(ROI) #this generates points from the function

# Training and classifiy

image = Kitui_after # defines the image for the Location
points = points_Kitui # defines the points to be used 
Kitui_Classified_After = Train_classify(image) 

Accuracy assesment 

In [ ]:
#Before image
image = Kitui_image_before
accuracy_Kitui = train_test(image)

confusionMatrix: 0.9384864427357346
Kappa: 0.9110189167009396


In [ ]:
# After image
image = Kitui_after
accuracy_Kitui = train_test(image)

confusionMatrix: 0.9323550465282748
Kappa: 0.8987139429301385


In [ ]:
# Export Images 
# geemap.ee_export_image_to_drive(Kitui_Classified_After, description='Kitui_Classified_After', folder='omdena', scale=10)
# geemap.ee_export_image_to_drive(Kitui_Classified_before, description='Kitui_Classified_before', folder='omdena', scale=10)

Laikipia and Samburu

In [ ]:
AOI = conv(Laikipia_samburu) # define AOI for the county
Laikipia_Samburu_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Laikipia_Samburu = training(ROI) #this generates points from the function

# Training and classifiy

image = Laikipia_Samburu_before_image # defines the image for the Location
points = points_Laikipia_Samburu # defines the points to be used 
Laikipia_Samburu_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Laikipia_Samburu_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Laikipia_Samburu_after_image # defines the image for the Location
points = points_Laikipia_Samburu # defines the points to be used 
Laikipia_Samburu_Classified_after = Train_classify(image)



Accuracy

In [ ]:
#Before image
image = Laikipia_Samburu_before_image
accuracy_Kitui = train_test(image)

confusionMatrix: 0.9192132068844397
Kappa: 0.8832877280738277


In [ ]:
#Atfer image
image = Laikipia_Samburu_after_image
accuracy_Laikipia_samburu = train_test(image)

confusionMatrix: 0.9261768901569187
Kappa: 0.8941371711074079


In [ ]:
#export the images
geemap.ee_export_image_to_drive(Laikipia_Samburu_Classified_after, description='Laikipia_samburu_Classified_After', folder='omdena', scale=10)
geemap.ee_export_image_to_drive(Laikipia_Samburu_Classified_before, description='Laikipia_samburu_Classified_before', folder='omdena', scale=10)

Machakos Embu

In [ ]:
AOI = conv(Machakos_embu) # define AOI for the county
Machakos_embu_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Machakos_embu = training(ROI) #this generates points from the function

# Training and classifiy

image = Machakos_embu_before_image # defines the image for the Location
points = points_Machakos_embu # defines the points to be used 
Machakos_embu_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Machakos_embu_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Machakos_embu_after_image # defines the image for the Location
points = points_Machakos_embu # defines the points to be used 
Machakos_embu_Classified_after = Train_classify(image)

Accuracy Assesment

In [ ]:
#Before image
image = Machakos_embu_before_image
accuracy_Machakos_embu = train_test(image)

confusionMatrix: 0.9508284339925174
Kappa: 0.8754053459338013


In [ ]:
# After image 
image = Machakos_embu_after_image
accuracy_Machakos_embu = train_test(image)

confusionMatrix: 0.9678558813140233
Kappa: 0.8932184751796715


In [ ]:
geemap.ee_export_image_to_drive(Machakos_embu_Classified_after, description='Machakos_Embu_Classified_After', folder='omdena', scale=10)
geemap.ee_export_image_to_drive(Machakos_embu_Classified_before, description='Machakos_Embu_Classified_before', folder='omdena', scale=10)

Mandera 

In [ ]:
AOI = conv(Mandera) # define AOI for the county
Mandera_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Mandera = training(ROI) #this generates points from the function

# Training and classifiy

image = Mandera_before_image # defines the image for the Location
points = points_Mandera # defines the points to be used 
Mandera_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Mandera_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Mandera_after_image # defines the image for the Location
points = points_Mandera # defines the points to be used 
Mandera_Classified_after = Train_classify(image)

Accuracy

In [ ]:
# Before Image
image = Mandera_before_image
accuracy_Kitui = train_test(image)


confusionMatrix: 0.9443466009158155
Kappa: 0.8847016215368679


In [ ]:
# After Image
image = Mandera_after_image
accuracy_Mandera = train_test(image)

confusionMatrix: 0.9379014989293362
Kappa: 0.8721960275358334


In [ ]:
#Export Images 
geemap.ee_export_image_to_drive(Mandera_Classified_after, description='Mandera_Classified_After', folder='omdena', scale=10)
geemap.ee_export_image_to_drive(Mandera_Classified_before, description='Mandera_Classified_before', folder='omdena', scale=10)

Marsabi

In [ ]:
AOI = conv(Marsabit) # define AOI for the county
Marsabit_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Marsabit = training(ROI) #this generates points from the function

# Training and classifiy

image = Marsabit_before_image # defines the image for the Location
points = points_Marsabit # defines the points to be used 
Marsabit_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Marsabit_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Marsabit_after_image # defines the image for the Location
points = points_Marsabit # defines the points to be used 
Marsabit_Classified_after = Train_classify(image)

Accuracy

In [ ]:
# Before Image
image = Marsabit_before_image
accuracy_Marsabit = train_test(image)

confusionMatrix: 0.9268640742062076
Kappa: 0.8896429140731381


In [ ]:
# After Image
image = Marsabit_after_image
accuracy_Marsabit = train_test(image)

confusionMatrix: 0.9265990912268438
Kappa: 0.8884205361482289


In [ ]:
#Export Images 
geemap.ee_export_image_to_drive(Marsabit_Classified_after, description='Marsabit_Classified_After', folder='omdena', scale=10)
geemap.ee_export_image_to_drive(Marsabit_Classified_before, description='Marsabit_Classified_before', folder='omdena', scale=10)

Meru Isiolo

In [ ]:
AOI = conv(Meru_isolo) # define AOI for the county
Meru_isolo_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Meru_isolo = training(ROI) #this generates points from the function

# Training and classifiy

image = Meru_isolo_before_image # defines the image for the Location
points = points_Meru_isolo # defines the points to be used 
Meru_isolo_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Meru_isolo_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Meru_isolo_after_image # defines the image for the Location
points = points_Meru_isolo # defines the points to be used 
Meru_isolo_Classified_after = Train_classify(image)

Accuracy

In [ ]:
# Before Image
image = Meru_isolo_before_image
accuracy_Meru_isiolo = train_test(image)

confusionMatrix: 0.9357015985790409
Kappa: 0.9036429094332422


In [ ]:
# After Image
image = Meru_isolo_after_image
accuracy_Meru_isiolo = train_test(image)

confusionMatrix: 0.9244617013766325
Kappa: 0.886686459930789


In [ ]:
geemap.ee_export_image_to_drive(Meru_isolo_Classified_after, description='Meru_isiolo_Classified_After', folder='omdena', scale=10)
geemap.ee_export_image_to_drive(Meru_isolo_Classified_before, description='Meru_isiolo_Classified_before', folder='omdena', scale=10)

Tana

In [ ]:
AOI = conv(Tana) # define AOI for the county
Tana_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Tana = training(ROI) #this generates points from the function

# Training and classifiy

image = Tana_before_image # defines the image for the Location
points = points_Tana # defines the points to be used 
Tana_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Tana_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Tana_after_image # defines the image for the Location
points = points_Tana # defines the points to be used 
Tana_Classified_after = Train_classify(image)

accuracy tana

In [ ]:
# Before Image
image = Tana_before_image
accuracy_Tana = train_test(image)

confusionMatrix: 0.9299264421215641
Kappa: 0.8636908316112087


In [ ]:
# After Image
image = Tana_after_image
accuracy_Tana = train_test(image)

confusionMatrix: 0.9202949438202247
Kappa: 0.8496766691399057


In [ ]:
AOI = conv(Turkana) # define AOI for the county
Turkana_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Turkana = training(ROI) #this generates points from the function

# Training and classifiy

image = Turkana_before_image # defines the image for the Location
points = points_Turkana # defines the points to be used 
Turkana_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Turkana_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Turkana_after_image # defines the image for the Location
points = points_Turkana # defines the points to be used 
Turkana_Classified_after = Train_classify(image)

Accuracy

In [ ]:
# Before Image
image = Turkana_before_image
accuracy_Turkana = train_test(image)

confusionMatrix: 0.9244060475161987
Kappa: 0.8856783402606717


In [ ]:
# After Image
image = Turkana_after_image
accuracy_Turkana = train_test(image)

confusionMatrix: 0.9207207207207208
Kappa: 0.8763590129216478


In [ ]:
# Export Images 
geemap.ee_export_image_to_drive(Tana_Classified_after, description='Tana_Classified_after', folder='omdena', scale=10)
geemap.ee_export_image_to_drive(Tana_Classified_before, description='Tana_Classified_before', folder='omdena', scale=10)
geemap.ee_export_image_to_drive(Turkana_Classified_after, description='Turkana_Classified_After', folder='omdena', scale=10)
geemap.ee_export_image_to_drive(Turkana_Classified_before, description='Turkana_Classified_before', folder='omdena', scale=10)

Wajir

In [ ]:
AOI = conv(Wajir) # define AOI for the county
Wajir_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_Wajir = training(ROI) #this generates points from the function

# Training and classifiy

image = Wajir_before_image # defines the image for the Location
points = points_Wajir # defines the points to be used 
Wajir_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
Wajir_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = Wajir_after_image # defines the image for the Location
points = points_Wajir # defines the points to be used 
Wajir_Classified_after = Train_classify(image)

accuracy Wajir

In [ ]:
# Before image
image = Wajir_before_image
accuracy_Wajir = train_test(image)

confusionMatrix: 0.9354838709677419
Kappa: 0.8790351499273157


In [ ]:
#After Image
image = Wajir_after_image
accuracy_Wajir = train_test(image)

confusionMatrix: 0.9367448471926084
Kappa: 0.8810176820667577


Tharaka Nithi

In [ ]:
tharaka = './drive/MyDrive/omdena/tharaka_shapefile/tharaka_nithi.shp'

In [ ]:

AOI = conv(tharaka) # define AOI for the county
tharaka_before_image = All_counties_before.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

ROI = AOI # define the Region to create points from the Sentinel 2 image 
points_tharaka = training(ROI) #this generates points from the function

# Training and classifiy

image = tharaka_before_image # defines the image for the Location
points = points_tharaka # defines the points to be used 
tharaka_Classified_before = Train_classify(image)

#after
#AOI = conv(Elgeiyo_Baringo) # define AOI for the county
tharaka_after_image = All_Counties_after.clip(AOI.geometry()) # Get the image for the location 
# Generation of trianing Points

#ROI = AOI # define the Region to create points from the Sentinel 2 image 
#points_Elgeiyo_baringo = training(ROI) #this generates points from the function

# Training and classifiy

image = tharaka_after_image # defines the image for the Location
points = points_tharaka # defines the points to be used 
tharaka_Classified_after = Train_classify(image)

accuracy Tharaka

In [ ]:
# Before image
image = tharaka_before_image
accuracy_Tharaka = train_test(image)

confusionMatrix: 0.9628040057224606
Kappa: 0.9143096849823584


In [ ]:
#After Image
image = tharaka_after_image
accuracy_Tharaka = train_test(image)

confusionMatrix: 0.9630028735632183
Kappa: 0.9153641294578885


In [ ]:
Map4 = geemap.Map()
legend_dict = {
    '1 Trees cover areas': '1c5f2c',
    '2 Shrubs cover areas': 'ccb879',
    '3 Grassland': 'dfdfc2',
    '4 Cropland': 'ab6c28',
    '5 Vegetation aquatic or regularly flooded': '6c9fb8',
    '6 Lichen Mosses / Sparse vegetation ': 'a3cc51',
    '7 Bare areas':'ab0000',
    '8 Built up areas': 'ab0000',
    '9 Snow and/or Ice': 'd1def8',
    '10 Open water': '466b9f'
}
viz_par = {'min': 1, 'max': 10,'palette': ['1c5f2c','ccb879','dfdfc2','ab6c28','6c9fb8','a3cc51','b3ac9f','ab0000','d1def8','466b9f' ]}
Map4.centerObject(AOI, 8)
Map4.addLayer(Garrisa_Classified_before, viz_par, 'Garissa Before')
Map4.addLayer(Garissa_Classified_After, viz_par, 'Garissa After attack Classified')
Map4.addLayer(Elgyo_Baringo_Classified_After, viz_par, 'Elgeiyo and Baringo After')
Map4.addLayer(Elgyo_Baringo_Classified_before, viz_par, 'Elgeiyo and Baringo Before')
Map4.addLayer(Kitui_Classified_After, viz_par, 'Kitui After')
Map4.addLayer(Kitui_Classified_before, viz_par, 'Kitui Before')
Map4.addLayer(Laikipia_Samburu_Classified_after, viz_par, 'Laikipia_samburu Classified After')
Map4.addLayer(Laikipia_Samburu_Classified_before, viz_par, 'Laikipia Samburu Classified Before')
Map4.addLayer(Machakos_embu_Classified_after, viz_par, 'Machakos Embu Classified After')
Map4.addLayer(Machakos_embu_Classified_before, viz_par, 'Machakos Embu Classified Before')
Map4.addLayer(Mandera_Classified_after, viz_par, 'Mandera Classified After')
Map4.addLayer(Mandera_Classified_before, viz_par, 'Mandera Classified  Before')
Map4.addLayer(Marsabit_Classified_after, viz_par, 'Marsabit Classified After')
Map4.addLayer(Marsabit_Classified_before, viz_par, 'Marsabit Classified  Before')
Map4.addLayer(Meru_isolo_Classified_after, viz_par, ' Meru_isiolo_Classified_After')
Map4.addLayer(Meru_isolo_Classified_before, viz_par, 'Meru_isiolo_Classified_Before')
Map4.addLayer(Tana_Classified_after, viz_par, ' Tana_Classified_After')
Map4.addLayer(Tana_Classified_before, viz_par, 'Tana_Classified_Before')
Map4.addLayer(Turkana_Classified_after, viz_par, ' Turkana_Classified_After')
Map4.addLayer(Turkana_Classified_before, viz_par, 'Turkana_Classified_Before')
Map4.addLayer(Wajir_Classified_after, viz_par, ' Wajir_Classified_After')
Map4.addLayer(Wajir_Classified_before, viz_par, 'Wajir_Classified_Before')
Map4.addLayer(tharaka_Classified_before,viz_par, 'Tharaka_Nithi_Classified_before')
Map4.add_layer(tharaka_Classified_after, viz_par, 'Tharaka_nithi_classified_after')
Map4.add_legend(legend_dict=legend_dict, title="LULC Affected Counties")
Map4.addLayerControl()

In [ ]:
Map4

In [ ]:
#Export tharaka
geemap.ee_export_image_to_drive(tharaka_Classified_after, description='Tharaka_Classified_after', folder='omdena', scale=10)
geemap.ee_export_image_to_drive(tharaka_Classified_before, description='Tharaka_Classified_before', folder='omdena', scale=10)

In [ ]:
#export Wajir
geemap.ee_export_image_to_drive(Wajir_after_image, description='Wajir_Classified_after', folder='omdena', scale=10)
geemap.ee_export_image_to_drive(Wajir_Classified_before, description='Wajir_Classified_before', folder='omdena', scale=10)


In [ ]:
Map4.save('LULC_DL_affected_counties_.html',close_file=True)